#Set-up

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
import tensorflow as tf
import tensorflow.contrib as tc
import tensorflow.contrib.layers as tcl
from tensorflow.contrib.layers import batch_norm
import time

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import pandas as pd
import numpy as np

#df = pd.read_csv("selected_users_data.csv")
#
#df = df[['lat','lon','alt','time-f']]
df = pd.read_pickle('original_79.pkl')
df["time-f"] = pd.to_datetime(df['time']).astype(int)/10**9
df = df[['lat','lon','time-f']]
df.head()

,lat,lon,time-f
0,39.975753,116.330313,1.274704e+09
1,39.975652,116.329943,1.274704e+09
2,39.975628,116.329563,1.274704e+09
3,39.975635,116.329460,1.274704e+09
4,39.975656,116.329468,1.274704e+09


In [ ]:
df.describe()

,lat,lon,time-f
count,11243.000000,11243.000000,1.124300e+04
mean,39.919573,116.373894,1.275613e+09
std,0.033148,0.039916,6.244559e+05
min,39.866934,116.318329,1.274704e+09
25%,39.895702,116.349771,1.275010e+09
50%,39.909312,116.357713,1.275537e+09
75%,39.941451,116.383709,1.276168e+09
max,39.989226,116.634624,1.276687e+09


In [ ]:
df

,lat,lon,time-f
0,39.975753,116.330313,1.274704e+09
1,39.975652,116.329943,1.274704e+09
2,39.975628,116.329563,1.274704e+09
3,39.975635,116.329460,1.274704e+09
4,39.975656,116.329468,1.274704e+09
...,...,...,...
326,39.879956,116.390871,1.276687e+09
327,39.879950,116.390843,1.276687e+09
328,39.879950,116.390768,1.276687e+09
329,39.879952,116.390711,1.276687e+09


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df2 = scaler.transform(df)
df2

array([[ 1.69486893, -1.09187489, -1.45582494],
       [ 1.69182189, -1.10114484, -1.45581694],
       [ 1.69109784, -1.11066533, -1.45580893],
       ...,
       [-1.19538006,  0.42275995,  1.71946107],
       [-1.19531973,  0.42133187,  1.71946908],
       [-1.19405264,  0.41774916,  1.71947708]])

#DP-GAN

In [ ]:
class Generator(object):
    def __init__(self, randomDim, l2scale, generatorDims, bn_train, generatorActivation, bnDecay, dataType):
        self.randomDim = randomDim
        self.l2scale = l2scale
        self.generatorDims = generatorDims
        self.bn_train = bn_train
        self.generatorActivation = generatorActivation
        self.bnDecay = bnDecay
        self.dataType = dataType
        self.name = 'geolife/g_net'

    def __call__(self, z, reuse=tf.AUTO_REUSE):
        tempVec = z
        tempDim = self.randomDim
        with tf.variable_scope(self.name, regularizer=tcl.l2_regularizer(self.l2scale), reuse=reuse):
            for i, genDim in enumerate(self.generatorDims[:-1]):
                W = tf.get_variable('W_' + str(i), shape=[tempDim, genDim])
                h = tf.matmul(tempVec, W)
                h2 = batch_norm(h) # GAN: batch_norm(h, decay=self.bnDecay, scale=True, is_training=self.bn_train, updates_collections=None)
                h3 = self.generatorActivation(h2)
                tempVec = h3 # GAN: + tempVec
                tempDim = genDim
            W = tf.get_variable('W' + str(i), shape=[tempDim, self.generatorDims[-1]])
            h = tf.matmul(tempVec, W)
            h2 = h # GAN: batch_norm(h, decay=self.bnDecay, scale=True, is_training=self.bn_train, updates_collections=None)

            if self.dataType == 'binary':
                h3 = tf.nn.sigmoid(h2) # GAN: tf.nn.tanh(h2)
            else:
                h3 = tf.nn.relu(h2)

            output = h3 # GAN: + tempVec
        return output

    @property
    def vars(self):
        return [var for var in tf.trainable_variables() if self.name in var.name]


class Discriminator(object):
    def __init__(self, inputDim, discriminatorDims, discriminatorActivation, l2scale):
        self.inputDim = inputDim
        self.discriminatorDims = discriminatorDims
        self.discriminatorActivation = discriminatorActivation
        self.l2scale = l2scale
        self.name = 'geolife/d_net'

    def __call__(self, x_input, reuse=tf.AUTO_REUSE):
        # batchSize = tf.shape(x_input)[0]
        # inputMean = tf.reshape(tf.tile(tf.reduce_mean(x_input, 0), [batchSize]), (batchSize, self.inputDim))
        # tempVec = tf.concat(axis = 1, values = [x_input, inputMean]) # https://stackoverflow.com/questions/41813665/tensorflow-slim-typeerror-expected-int32-got-list-containing-tensors-of-type
        # tempDim = self.inputDim * 2 # need in GAN
        tempVec = x_input
        tempDim = self.inputDim # remove in GAN
        with tf.variable_scope(self.name, reuse=reuse): # GAN: regularizer=tcl.l2_regularizer(self.l2scale)
            for i, discDim in enumerate(self.discriminatorDims[:-1]):
                W = tf.get_variable('W_' + str(i), shape=[tempDim, discDim])
                b = tf.get_variable('b_' + str(i), shape=[discDim])
                h = self.discriminatorActivation(tf.add(tf.matmul(tempVec, W), b))
                # h = tf.nn.dropout(h, keepRate) # need in GAN
                tempVec = h
                tempDim = discDim
            W = tf.get_variable('W', shape=[tempDim, 1])
            b = tf.get_variable('b', shape=[1])
            y_hat = tf.squeeze(tf.add(tf.matmul(tempVec, W), b)) # need sigmoid in GAN

        return y_hat

    @property
    def vars(self):
        return [var for var in tf.global_variables() if self.name in var.name]

In [ ]:
class WassersteinGAN(object):
    def __init__(self, g_net, d_net, z_sampler, data, batch_size=64): # changed
        self.trainX = data
        self.g_net = g_net
        self.d_net = d_net
        self.z_sampler = z_sampler
        self.x_dim = self.d_net.inputDim
        self.z_dim = self.g_net.randomDim
        # self.x = tf.placeholder(tf.float32, [None] + self.x_dim, name='x')
        # self.z = tf.placeholder(tf.float32, [None, self.z_dim], name='z')
        self.x = tf.placeholder(tf.float32, [None, self.x_dim], name='x')
        self.z = tf.placeholder(tf.float32, [None, self.z_dim], name='z')

        self.x_ = self.g_net(self.z)

        self.d = self.d_net(self.x)
        self.d_ = self.d_net(self.x_)

        self.g_loss = tf.reduce_mean(self.d_)
        self.d_loss = tf.reduce_mean(self.d) - tf.reduce_mean(self.d_)

        self.reg = tc.layers.apply_regularization(
            tc.layers.l1_regularizer(2.5e-5),
            weights_list=[var for var in tf.global_variables() if 'W' in var.name]
        )
        self.g_loss_reg = self.g_loss + self.reg
        self.d_loss_reg = self.d_loss + self.reg

        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_rmsprop = tf.train.RMSPropOptimizer(learning_rate=5e-5)  # DP case
            grads_and_vars = self.d_rmsprop.compute_gradients(-1*self.d_loss_reg, var_list=self.d_net.vars)
            dp_grads_and_vars = []  # noisy version
            for gv in grads_and_vars:  # for each pair
                g = gv[0]  # get the gradient, type in loop one: Tensor("gradients/AddN_37:0", shape=(4, 4, 1, 64), dtype=float32)
                #print g # shape of all vars
                if g is not None:  # skip None case
                    g = self.dpnoise(g, batch_size)  # add noise on the tensor, type in loop one: Tensor("Add:0", shape=(4, 4, 1, 64), dtype=float32)
                dp_grads_and_vars.append((g, gv[1]))
            self.d_rmsprop_new = self.d_rmsprop.apply_gradients(dp_grads_and_vars) # should assign to a new optimizer
            # self.d_rmsprop = tf.train.RMSPropOptimizer(learning_rate=5e-5) \
            #     .minimize(-1*self.d_loss_reg, var_list=self.d_net.vars) # non-DP case
            self.g_rmsprop = tf.train.RMSPropOptimizer(learning_rate=5e-5) \
                .minimize(-1*self.g_loss_reg, var_list=self.g_net.vars)

        self.d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in self.d_net.vars]
        self.d_net_var_grad = [i for i in tf.gradients(self.d_loss_reg, self.d_net.vars) if i is not None] # explore the effect of noise on norm of D net variables's gradient vector, also remove None type
        self.norm_d_net_var_grad = []
        gpu_options = tf.GPUOptions(allow_growth=True)
        self.sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
        # self.sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True))
        self.g_loss_store = []  # store loss of generator
        self.d_loss_store = [] # store loss of discriminator
        self.wdis_store = []  # store Wasserstein distance, new added
        # print self.d_net.vars
        # print self.g_net.vars

    def train(self, nEpochs, batchSize):
        start_time = time.time()
        self.nBatches = int(np.ceil(float(self.trainX.shape[0]) / float(batchSize)))
        self.sess.run(tf.initialize_all_variables())
        for epoch in range(nEpochs):
            d_iters = 2
            for i in range(self.nBatches):
                rd_loss = 0
                rg_loss = 0
                for _ in range(d_iters): # train discriminator
                    batchX = self.trainX[np.random.randint(low=0,high=self.trainX.shape[0],size=batchSize)]
                    randomZ = self.z_sampler(batchSize, self.z_dim)
                    _, rd_loss = self.sess.run([self.d_rmsprop_new, self.d_loss], feed_dict={self.x: batchX, self.z: randomZ}) # DP case
                    self.sess.run(self.d_clip)

                randomZ = self.z_sampler(batchSize, self.z_dim) # train generator
                _, rg_loss = self.sess.run([self.g_rmsprop, self.g_loss], feed_dict={self.x: batchX, self.z: randomZ})

                if i % 50 == 0: # print out loss
                    print('Iter [%5d] Time [%5.4f] d_loss [%.6f] g_loss [%.6f]' %
                          (epoch, time.time() - start_time, rd_loss, rg_loss))

                # store rd_loss and rg_loss, new added
                self.g_loss_store.append(rg_loss)  # g_loss will decrease, here is not self.g_loss nor self.g_loss_reg
                self.d_loss_store.append(rd_loss)  # d_loss will increase
                self.wdis_store.append(-1*rd_loss)  # Wasserstein distance will decrease

        z_sample = self.z_sampler(self.trainX.shape[0], self.z_dim) # generate EHR from generator, after finish training
        x_gene = self.sess.run(self.x_, feed_dict={self.z: z_sample})

        return self.trainX, x_gene

    def dpnoise(self, tensor, batch_size):
        '''add noise to tensor'''
        s = tensor.get_shape().as_list()  # get shape of the tensor
        sigma = 0.00001  # assign it manually
        cg = 160000.0
        rt = tf.random_normal(s, mean=0.0, stddev=sigma * cg)
        t = tf.add(tensor, tf.scalar_mul((1.0 / batch_size), rt))
        return t

    def find(self, gen, train):
        dist = []
        for i in range(len(train)):
            dist.append(linalg.norm(array(gen) - array(train[i])))
        return argmin(dist)

    def norm_w(self, v):
        return sum([linalg.norm(i) for i in v])


In [ ]:
class NoiseSampler(object):
    def __call__(self, batch_size, z_dim):
        return np.random.normal(size=(batch_size, z_dim))

In [ ]:
dataType = 'binary'
#inputDim = 4
inputDim = 3
randomDim = 100
generatorDims = list((128, 64)) + [inputDim]
#discriminatorDims = (16, 4, 1)
discriminatorDims = (16, 3, 1)
bnDecay = 0.99
l2scale = 2.5e-5 # WGAN: 2.5e-5, GAN: 0.001
nEpochs = 1000
batchSize = 1024
cilpc = 0.01
n_discriminator_update = 2
learning_rate = 5e-4 # GAN: 0.001
adj = 1.0
db = 0.5
bn_train = True
_VALIDATION_RATIO = 0.25
top = 1071 # 1071 for original data, other: 1071 (in paper), 512, 64
if dataType == 'binary':
    aeActivation = tf.nn.tanh
else:
    aeActivation = tf.nn.relu
generatorActivation = tf.nn.relu
discriminatorActivation = tf.nn.relu

zs = NoiseSampler()
g_net = Generator(randomDim, l2scale, generatorDims, bn_train, generatorActivation, bnDecay, dataType)
d_net = Discriminator(inputDim, discriminatorDims, discriminatorActivation, l2scale)

In [ ]:
wgan = WassersteinGAN(g_net, d_net, zs, df2)
x_train, x_gene = wgan.train(nEpochs, batchSize)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter [    0] Time [0.3237] d_loss [-0.000000] g_loss [0.009896]
Iter [    1] Time [0.7084] d_loss [-0.000000] g_loss [0.009826]
Iter [    2] Time [1.0860] d_loss [-0.000000] g_loss [0.009706]
Iter [    3] Time [1.4634] d_loss [-0.000000] g_loss [0.009795]
Iter [    4] Time [1.8440] d_loss [-0.000000] g_loss [0.009856]
Iter [    5] Time [2.2020] d_loss [-0.000000] g_loss [0.009809]
Iter [    6] Time [2.5786] d_loss [-0.000000] g_loss [0.009867]
Iter [    7] Time [2.9300] d_loss [0.000000] g_loss [0.009714]
Iter [    8] Time [3.2844] d_loss [-0.000000] g_loss [0.009670]
Iter [    9] Time [3.6497] d_loss [-0.000000] g_loss [0.009674]
Iter [   10] Time [4.0286] d_loss [0.000000] g_loss [0.009905]
Iter [   11] Time [4.4148] d_loss [-0.000000] g_loss [0.009819]
Iter [   12] Time [4.8320] d_loss [-0.0000

In [ ]:
pd.DataFrame(x_train).describe().apply(round, args=([6]))

,0,1,2
count,11243.000000,11243.000000,11243.000000
mean,-0.000000,0.000000,-0.000000
std,1.000044,1.000044,1.000044
min,-1.588055,-1.392121,-1.455825
25%,-0.720178,-0.604388,-0.965329
50%,-0.309568,-0.405397,-0.121995
75%,0.660038,0.245904,0.888912
max,2.101331,6.532309,1.719477


In [ ]:
pd.DataFrame(x_gene).describe()

,0,1,2
count,11243.000000,11243.000000,11243.000000
mean,0.382846,0.512418,0.813868
std,0.358772,0.351951,0.281953
min,0.000005,0.000019,0.000110
25%,0.045086,0.160201,0.751617
50%,0.252845,0.525199,0.966803
75%,0.735247,0.868768,0.996410
max,0.999987,0.999981,1.000000


#Preparate Synthetic data to MMC

In [ ]:
df_train2 = scaler.inverse_transform(x_gene)
df_train2 = pd.DataFrame(df_train2)
df_train2.rename(columns={0:'lat',
                          1:'lon',
                          2:'time'}, 
                 inplace=True)
#df_train2

df_train2['time'] = pd.to_datetime(df_train2['time'], unit='s')
df_train2['user'] = 79
df_train2 = df_train2[['user','time','lat','lon']]
df_train2

,user,time,lat,lon
0,79,2010-06-06 23:02:24,39.950657,116.379280
1,79,2010-06-11 06:00:32,39.931725,116.408440
2,79,2010-06-11 05:56:16,39.921555,116.403030
3,79,2010-06-11 06:11:12,39.939987,116.382111
4,79,2010-06-11 06:21:52,39.946987,116.392799
...,...,...,...,...
11238,79,2010-06-11 01:21:04,39.928604,116.381927
11239,79,2010-06-11 03:46:08,39.920189,116.375633
11240,79,2010-06-09 18:25:04,39.921635,116.397972
11241,79,2010-06-09 18:12:16,39.943844,116.410820


In [ ]:
df_train = scaler.inverse_transform(x_train)
df_train = pd.DataFrame(df_train)
df_train.rename(columns={0:'lat',
                          1:'lon',
                          2:'time'}, 
                 inplace=True)
#df_train

df_train['time'] = pd.to_datetime(df_train['time'], unit='s')
df_train['user'] = 79
df_train = df_train[['user','time','lat','lon']]
df_train

,user,time,lat,lon
0,79,2010-05-24 12:27:28,39.975753,116.330313
1,79,2010-05-24 12:27:33,39.975652,116.329943
2,79,2010-05-24 12:27:38,39.975628,116.329563
3,79,2010-05-24 12:27:43,39.975635,116.329460
4,79,2010-05-24 12:27:48,39.975656,116.329468
...,...,...,...,...
11238,79,2010-06-16 11:12:56,39.879956,116.390871
11239,79,2010-06-16 11:13:01,39.879950,116.390843
11240,79,2010-06-16 11:13:06,39.879950,116.390768
11241,79,2010-06-16 11:13:11,39.879952,116.390711


In [ ]:
df_train2.to_csv('dpgan2_79.csv', index=False, header=None)
df_train.to_csv('dpgan1_79.csv', index=False, header=None)

In [ ]:
df_train2.describe()

,user,lat,lon
count,11243.0,11243.000000,11243.000000
mean,79.0,39.930790,116.389656
std,0.0,0.011983,0.014810
min,79.0,39.919575,116.373894
25%,79.0,39.921068,116.380287
50%,79.0,39.927956,116.394859
75%,79.0,39.943945,116.408569
max,79.0,39.952721,116.413811


In [ ]:
df_train.describe()

,user,lat,lon
count,11243.0,11243.000000,11243.000000
mean,79.0,39.919573,116.373894
std,0.0,0.033148,0.039916
min,79.0,39.866934,116.318329
25%,79.0,39.895702,116.349771
50%,79.0,39.909312,116.357713
75%,79.0,39.941451,116.383709
max,79.0,39.989226,116.634624
